In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from torch import nn
import lightning as L
from torch.utils.data import DataLoader, Dataset
from lightning import LightningModule, Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torchmetrics.functional import accuracy
import random
import re


%load_ext autoreload
%autoreload 2
%load_ext rich

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rich extension is already loaded. To reload it, use:
  %reload_ext rich


# Prepare Data

## Preprocess data

### Character-level model

In [120]:
class ShakespeareDataset(Dataset):
    def __init__(self, text, seq_length=100):
        self.chars = sorted(list(set(text)))
        self.char_to_int = {ch: i for i, ch in enumerate(self.chars)}
        self.int_to_char = {i: ch for i, ch in enumerate(self.chars)}
        self.data_size, self.vocab_size = len(text), len(self.chars)
        self.seq_length = seq_length
        # self.embedding_dim = L.Embedding(num_embeddings=self.vocab_size, embedding_dim=256)

        # Create training data
        self.x = []
        self.y = []
        for i in range(0, self.data_size - seq_length, 1):
            seq_in = text[i:i + seq_length]
            seq_out = text[i + seq_length]
            self.x.append([self.char_to_int[char] for char in seq_in])
            self.y.append(self.char_to_int[seq_out])
        self.x = np.array(self.x)
        self.y = np.array(self.y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return torch.tensor(self.x[index], dtype=torch.long), torch.tensor(self.y[index], dtype=torch.long)

## Load dataset

In [121]:
def load_data(file_path, seq_length=100, train_split=0.7, valid_split=0.15):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    dataset = ShakespeareDataset(text, seq_length)
    train_size = int(len(dataset) * train_split)
    valid_size = int(len(dataset) * valid_split)
    test_size = len(dataset) - train_size - valid_size
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
    return dataset, train_dataset, valid_dataset, test_dataset

# Model Definition

In [122]:
class TextGenerator(L.LightningModule):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, layers, dropout=0.2):
        super().__init__()
        self.save_hyperparameters()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x

    def training_step(self, batch):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        self.log('train_loss', loss, logger=True, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        self.log('val_loss', loss, logger=True, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [123]:
# Load your datasets
dataset, train_dataset, valid_dataset, test_dataset = load_data('input.txt', seq_length=100)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [124]:
# Create the model
model = TextGenerator(len(train_dataset.dataset.chars), embedding_dim=128, hidden_dim=256, layers=2)

# Setup trainer and fit the model using Lightning's Trainer
trainer = L.Trainer(max_epochs=10, callbacks=[ModelCheckpoint(monitor="val_loss"), EarlyStopping(monitor="val_loss")])
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 8.3 K 
1 | lstm      | LSTM      | 921 K 
2 | fc        | Linear    | 16.7 K
----------------------------------------
946 K     Trainable params
0         Non-trainable params
946 K     Total params
3.786     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [125]:
def generate_text(model, start_str, gen_length=100, temperature=1.0):
    model.eval()
    input_seq = [dataset.char_to_int[c] for c in start_str[-dataset.seq_length:]]
    input_seq = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(model.device)

    text = start_str
    hidden = None

    for _ in range(gen_length):
        output, hidden = model.lstm(model.embedding(input_seq), hidden)
        output_logits = model.fc(output[:, -1, :])
        p = torch.nn.functional.softmax(output_logits / temperature, dim=-1).detach().cpu().numpy().squeeze()
        char_ind = np.random.choice(len(dataset.chars), p=p)
        next_char = dataset.int_to_char[char_ind]
        text += next_char

        input_seq = torch.cat((input_seq[:, 1:], torch.tensor([[char_ind]], dtype=torch.long).to(model.device)), dim=1)

    return text

    
print(generate_text(model, 'ROMEO:', gen_length=1000, temperature=1.0))

ROMEO:
Look, I do,
Fie, Hearts:
A same with he;
And, then, stayd;
There your divook,
Our live;
Stream theirs;
Go there in
This of hells,
Here in there,
That told us incluked.

ANGEL:
My lord,
Alies, beseeching,
Holt, he sayel,
He was; would
grace, for on
Is leave grandling.

BUCKINGUM:
Live my appared.

ROMEL:
With fearfullies:
Now I expect.
Therefory land.
Welcome looks
Another and heavy
That kings,
With away.

WARDIY:
Not Gremion,
Why whom senten'd so;
There's here,
To be verse.
That apt stom
Anon, cannot
Some of Aufirs:
Lend us to ingening from the deep,
His drunk.

BISTAND:
Your root!

DUCHES:
But depose vessel,
A beauty,
Or thy satisful
That thou will.
Though withdread,
Brougn with thy weepings soon well,
Let ten tride.

MARCIUS:
As see't,--

FERDINAL:
I have from duke,
With for stood.
How nothing?

PETRUCHI:
But it it,
Busition he meet,
Leakelo up,
Where fivour's like
All thus red then;
Your bond,
wounds, heard!
Wherew that I to Rome?
Harp, soldied
counter, well,
A plant:
Ay, ill